In [14]:
#### MICHIGAN STATE EXPLORE DATA SCRATCHBOOK 

## 1. Import Libraries
# Dependencies
import pandas as pd
import sqlite3

## Datafile - Current Season

# path = 'NEW2022-2023 College Hockey Data.db'
path = "FRIDAY_NEW 2020-2021 Season Data.db"
path = "FRIDAY_NEW 2018-2019 Season Data.db"
## Folder: ../workbook/db_files_raw




path = '../workbook/db_files_raw/2022 Game Stats.db'

db_path = path # for use by the MSU Specific SQL queries
TAG = "2022 Season"

# Load the database
conn = sqlite3.connect(path)


# Get the list of tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
table_names = [table[0] for table in tables]

# table_names

# Penelties

## First Create a dictionary of the classes of penelty


### Penelty Classification

- Physical Aggressiveness: Boarding, Charging, Elbowing, Roughing
- Stick Infractions: Slashing, High-Sticking, Hooking, Holding the Stick
- Game Flow Disruptions: Delay of Game, Unsportsmanlike Conduct, Interference
- Illegal Contact: Kneeing, Tripping
- Contact to the Head: Contact to the Head

We'll use these subcategories along with the broader categories like Minor, Major, Misconduct, and Match Penalties to create a multi-level classification.


In [15]:
# Creating a Python dictionary to map each penalty type to its corresponding category and subcategory

# Subcategories for Minor Penalties
minor_subcategories = {
    'Physical Aggressiveness': ['Boarding', 'Charging', 'Elbowing', 'Roughing'],
    'Stick Infractions': ['Slashing', 'High-Sticking', 'Hooking', 'Holding the Stick'],
    'Game Flow Disruptions': ['Delay of Game', 'Unsportsmanlike Conduct', 'Interference'],
    'Illegal Contact': ['Kneeing', 'Tripping'],
    'Contact to the Head': ['Contact to the Head']
}

# Other broader categories (for illustration, actual list should be based on the data)
major_penalties = ['Butt-Ending', 'Checking from Behind', 'Fighting', 'Fight Instigator', 'Holding/Grasping the Facemask', 'Pushing-off of Opponent with Skate', 'Leaving the Bench During an Altercation', 'Spearing']
misconduct_penalties = ['Misconduct']  # Placeholder, actual list should be based on the data
match_penalties = ['Match Penalty']  # Placeholder, actual list should be based on the data

# Initialize the mapping dictionary
penalty_mapping = {}

# Fill in the dictionary for minor penalties
for subcategory, penalties in minor_subcategories.items():
    for penalty in penalties:
        penalty_mapping[penalty] = {'Category': 'Minor', 'Subcategory': subcategory}

# Fill in the dictionary for major, misconduct, and match penalties (without subcategories)
for penalty in major_penalties:
    penalty_mapping[penalty] = {'Category': 'Major', 'Subcategory': None}
for penalty in misconduct_penalties:
    penalty_mapping[penalty] = {'Category': 'Misconduct', 'Subcategory': None}
for penalty in match_penalties:
    penalty_mapping[penalty] = {'Category': 'Match', 'Subcategory': None}

# Show the penalty mapping dictionary
# penalty_mapping



## By Referee Across entire dataset


In [16]:
# Fetching data from the game_details and penalty_summary tables
query_game_details_data = "SELECT * FROM game_details;"
query_penalty_summary_data = "SELECT * FROM penalty_summary;"

# Execute the queries and fetch results into DataFrames
df_game_details = pd.read_sql(query_game_details_data, conn)
df_penalty_summary = pd.read_sql(query_penalty_summary_data, conn)

# Merge the DataFrames on Game_ID to get referee information along with penalties
df_merged = pd.merge(df_penalty_summary, df_game_details, on='Game_ID')

# Add the penalty classifications (assuming the mapping dictionary 'penalty_mapping' is already defined)
df_merged['Category'] = df_merged['Penalty_Type'].map(lambda x: penalty_mapping.get(x, {}).get('Category', 'Unknown'))
df_merged['Subcategory'] = df_merged['Penalty_Type'].map(lambda x: penalty_mapping.get(x, {}).get('Subcategory', 'Unknown'))


# Show the first few rows of the merged DataFrame to verify
# df_merged.head()

# Count the number of games each referee has worked
game_count_ref1 = df_merged.groupby('Ref1')['Game_ID'].nunique().reset_index().rename(columns={'Ref1': 'Referee', 'Game_ID': 'GameCount'})
game_count_ref2 = df_merged.groupby('Ref2')['Game_ID'].nunique().reset_index().rename(columns={'Ref2': 'Referee', 'Game_ID': 'GameCount'})
game_count = pd.concat([game_count_ref1, game_count_ref2]).groupby('Referee').sum().reset_index()

# Calculate the total number of penalty minutes each referee has called
penalty_minutes_ref1 = df_merged.groupby('Ref1')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Ref1': 'Referee', 'Pen_Length': 'TotalPenaltyMinutes'})
penalty_minutes_ref2 = df_merged.groupby('Ref2')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Ref2': 'Referee', 'Pen_Length': 'TotalPenaltyMinutes'})
penalty_minutes = pd.concat([penalty_minutes_ref1, penalty_minutes_ref2]).groupby('Referee').sum().reset_index()

# Calculate the total count of penalties each referee has called
penalty_count_ref1 = df_merged.groupby('Ref1').size().reset_index().rename(columns={0: 'TotalPenaltyCount', 'Ref1': 'Referee'})
penalty_count_ref2 = df_merged.groupby('Ref2').size().reset_index().rename(columns={0: 'TotalPenaltyCount', 'Ref2': 'Referee'})
penalty_count = pd.concat([penalty_count_ref1, penalty_count_ref2]).groupby('Referee').sum().reset_index()

# Merge all the aggregated DataFrames
df_ref_stats_python = pd.merge(game_count, penalty_minutes, on='Referee', how='outer')
df_ref_stats_python = pd.merge(df_ref_stats_python, penalty_count, on='Referee', how='outer')

# Calculate the average number of penalties and penalty minutes per game for each referee
df_ref_stats_python['AvgPenaltyMinutesPerGame'] = df_ref_stats_python['TotalPenaltyMinutes'] / df_ref_stats_python['GameCount']
df_ref_stats_python['AvgPenaltyCountPerGame'] = df_ref_stats_python['TotalPenaltyCount'] / df_ref_stats_python['GameCount']

# Sort by average penalty count per game
df_ref_stats_python = df_ref_stats_python.sort_values(by='AvgPenaltyCountPerGame', ascending=False).reset_index(drop=True)


# Identifying the unique types of penalties in the data
unique_penalty_types = df_merged['Penalty_Type'].unique()

# Initialize an empty DataFrame to hold the aggregated data
df_penalty_type_breakdown = pd.DataFrame()

# Loop through each unique penalty type to calculate the total minutes for each referee
for penalty_type in unique_penalty_types:
    penalty_minutes_ref1 = df_merged[df_merged['Penalty_Type'] == penalty_type].groupby('Ref1')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Ref1': 'Referee', 'Pen_Length': f"{penalty_type}_Minutes"})
    penalty_minutes_ref2 = df_merged[df_merged['Penalty_Type'] == penalty_type].groupby('Ref2')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Ref2': 'Referee', 'Pen_Length': f"{penalty_type}_Minutes"})
    penalty_minutes = pd.concat([penalty_minutes_ref1, penalty_minutes_ref2]).groupby('Referee').sum().reset_index()
    
    # Merge with the existing DataFrame
    if df_penalty_type_breakdown.empty:
        df_penalty_type_breakdown = penalty_minutes
    else:
        df_penalty_type_breakdown = pd.merge(df_penalty_type_breakdown, penalty_minutes, on='Referee', how='outer')


##############################################
## Filter out referees with less than 10 games
# df_ref_stats_python = df_ref_stats_python[df_ref_stats_python['GameCount'] >= 10].reset_index(drop=True)
##################################################


# Merge the penalty type breakdown with the main referee stats DataFrame
df_ref_stats_extended = pd.merge(df_ref_stats_python, df_penalty_type_breakdown, on='Referee', how='outer')


# Sort by Game Count
df_ref_stats_extended = df_ref_stats_extended.sort_values(by='GameCount', ascending=False).reset_index(drop=True)


# Show the top of the DataFrame
df_ref_stats_extended.sample(10)


,Referee,GameCount,TotalPenaltyMinutes,TotalPenaltyCount,AvgPenaltyMinutesPerGame,AvgPenaltyCountPerGame,Interference_Minutes,Tripping_Minutes,Cross-Checking_Minutes,Slashing_Minutes,...,Brokenstick_Minutes,Persistinginmisconduct_Minutes,Shot After the Whistle_Minutes,Clipping_Minutes,Illegal Equipment_Minutes,Kicking_Minutes,Excessiveroughness_Minutes,Bodychecking_Minutes,Taunting_Minutes,Butt-ending_Minutes
17,Ryan Hersey,24,657,253,27.375000,10.541667,72.0,96.0,55.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,John Melancon,5,116,51,23.200000,10.200000,20.0,20.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,Morello,1,14,7,14.000000,7.000000,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,Brady Johnson,14,221,106,15.785714,7.571429,14.0,28.0,23.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,Daniel Kovarik,9,127,59,14.111111,6.555556,16.0,16.0,10.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,Bobby Lukkason,11,323,105,29.363636,9.545455,26.0,32.0,24.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Geoff Miller,9,124,62,13.777778,6.888889,12.0,34.0,6.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,BRAD ALBERS,1,40,17,40.000000,17.000000,4.0,4.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Nick Bet,2,40,20,20.000000,10.000000,4.0,6.0,6.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,Walker Holton,2,24,12,12.000000,6.000000,2.0,8.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Function to aggregate referee stats based on new penalty classifications
def aggregate_referee_stats(df):
    # Initialize an empty DataFrame to store the final aggregated stats
    df_aggregated = pd.DataFrame()

    # List to store the types of aggregations (categories and subcategories)
    aggregation_types = df['Category'].unique().tolist() + df['Subcategory'].unique().tolist()
    aggregation_types = [x for x in aggregation_types if x != 'Unknown']

    # Loop through each aggregation type to perform the calculations
    for agg_type in aggregation_types:
        # Filtering data based on the aggregation type
        df_filtered = df[(df['Category'] == agg_type) | (df['Subcategory'] == agg_type)]

        # Calculating the aggregated stats for referees in Ref1 and Ref2 positions
        ref1_stats = df_filtered.groupby('Ref1')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Ref1': 'Referee', 'Pen_Length': f"{agg_type}_Minutes"})
        ref2_stats = df_filtered.groupby('Ref2')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Ref2': 'Referee', 'Pen_Length': f"{agg_type}_Minutes"})
        
        # Concatenating and summing up the stats for each referee
        aggregated_stats = pd.concat([ref1_stats, ref2_stats]).groupby('Referee').sum().reset_index()
        
        # Merging with the existing DataFrame
        if df_aggregated.empty:
            df_aggregated = aggregated_stats
        else:
            df_aggregated = pd.merge(df_aggregated, aggregated_stats, on='Referee', how='outer')
    
    return df_aggregated

# Using the function to aggregate the referee stats based on the new penalty classifications
df_ref_stats_updated = aggregate_referee_stats(df_merged)

# Merging with the main referee stats DataFrame
df_ref_stats_updated = pd.merge(df_ref_stats_python, df_ref_stats_updated, on='Referee', how='outer')

# drop the unnessecary original penalty columns

##############################################
## Filter out referees with less than 10 games
# df_ref_stats_updated = df_ref_stats_updated[df_ref_stats_updated['GameCount'] >= 10].reset_index(drop=True)
##################################################


# Show the top of the DataFrame
# df_ref_stats_updated.head()


## Team by Team summary 

In [18]:


# Count the number of games each team has played
game_count_team = df_merged.groupby('Team')['Game_ID'].nunique().reset_index().rename(columns={'Game_ID': 'GameCount'})

# Calculate the total number of penalty minutes for each team
total_penalty_minutes_team = df_merged.groupby('Team')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Pen_Length': 'TotalPenaltyMinutes'})

# Calculate the total count of penalties for each team
total_penalty_count_team = df_merged.groupby('Team').size().reset_index().rename(columns={0: 'TotalPenaltyCount'})

# Count the number of different players from each team who have been penalized
unique_players_penalized_team = df_merged.groupby('Team')['Player'].nunique().reset_index().rename(columns={'Player': 'UniquePlayersPenalized'})

# Initialize an empty DataFrame to hold the penalty type breakdown
df_team_penalty_type_breakdown = pd.DataFrame()

# Loop through each unique penalty type to calculate the total minutes for each team
for penalty_type in unique_penalty_types:
    team_penalty_minutes = df_merged[df_merged['Penalty_Type'] == penalty_type].groupby('Team')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Pen_Length': f"{penalty_type}_Minutes"})
    
    # Merge with the existing DataFrame
    if df_team_penalty_type_breakdown.empty:
        df_team_penalty_type_breakdown = team_penalty_minutes
    else:
        df_team_penalty_type_breakdown = pd.merge(df_team_penalty_type_breakdown, team_penalty_minutes, on='Team', how='outer')

# Merge all the aggregated DataFrames to create the final team stats DataFrame
df_team_stats = pd.merge(game_count_team, total_penalty_minutes_team, on='Team', how='outer')
df_team_stats = pd.merge(df_team_stats, total_penalty_count_team, on='Team', how='outer')
df_team_stats = pd.merge(df_team_stats, unique_players_penalized_team, on='Team', how='outer')
df_team_stats = pd.merge(df_team_stats, df_team_penalty_type_breakdown, on='Team', how='outer')

# Calculate the average number of penalties and penalty minutes per game for each team
df_team_stats['AvgPenaltyMinutesPerGame'] = df_team_stats['TotalPenaltyMinutes'] / df_team_stats['GameCount']
df_team_stats['AvgPenaltyCountPerGame'] = df_team_stats['TotalPenaltyCount'] / df_team_stats['GameCount']

# Show the top of the DataFrame
# df_team_stats.head()


In [19]:
# Function to aggregate team stats based on new penalty classifications
def aggregate_team_stats(df):
    # Initialize an empty DataFrame to store the final aggregated stats
    df_aggregated = pd.DataFrame()

    # List to store the types of aggregations (categories and subcategories)
    aggregation_types = df['Category'].unique().tolist() + df['Subcategory'].unique().tolist()
    aggregation_types = [x for x in aggregation_types if x != 'Unknown']

    # Loop through each aggregation type to perform the calculations
    for agg_type in aggregation_types:
        # Filtering data based on the aggregation type
        df_filtered = df[(df['Category'] == agg_type) | (df['Subcategory'] == agg_type)]

        # Calculating the aggregated stats for each team
        team_stats = df_filtered.groupby('Team')['Pen_Length'].apply(lambda x: x.astype(int).sum()).reset_index().rename(columns={'Pen_Length': f"{agg_type}_Minutes"})
        
        # Merging with the existing DataFrame
        if df_aggregated.empty:
            df_aggregated = team_stats
        else:
            df_aggregated = pd.merge(df_aggregated, team_stats, on='Team', how='outer')
    
    return df_aggregated

# Using the function to aggregate the team stats based on the new penalty classifications
df_team_stats_updated = aggregate_team_stats(df_merged)

# Merge all the aggregated DataFrames to create the final team stats DataFrame
df_team_stats_final = pd.merge(game_count_team, total_penalty_minutes_team, on='Team', how='outer')
df_team_stats_final = pd.merge(df_team_stats_final, total_penalty_count_team, on='Team', how='outer')
df_team_stats_final = pd.merge(df_team_stats_final, unique_players_penalized_team, on='Team', how='outer')
df_team_stats_final = pd.merge(df_team_stats_final, df_team_stats_updated, on='Team', how='outer')

# Calculate the average number of penalties and penalty minutes per game for each team
df_team_stats_final['AvgPenaltyMinutesPerGame'] = df_team_stats_final['TotalPenaltyMinutes'] / df_team_stats_final['GameCount']
df_team_stats_final['AvgPenaltyCountPerGame'] = df_team_stats_final['TotalPenaltyCount'] / df_team_stats_final['GameCount']

# Dropping the old columns that are redundant with the new aggregation
# Since we've named the new columns differently, no old columns should be redundant in this case

# Show the top of the DataFrame
df_team_stats_final.head()




,Team,GameCount,TotalPenaltyMinutes,TotalPenaltyCount,UniquePlayersPenalized,Minor_Minutes,Misconduct_Minutes,Major_Minutes,Game Flow Disruptions_Minutes,Illegal Contact_Minutes,Stick Infractions_Minutes,Contact to the Head_Minutes,Physical Aggressiveness_Minutes,None_Minutes,AvgPenaltyMinutesPerGame,AvgPenaltyCountPerGame
0,AIC,34,329,131,28,188,NaN,5.0,54,26,56,10.0,42,NaN,9.676471,3.852941
1,ASU,39,564,196,26,298,30.0,10.0,62,44,80,15.0,97,NaN,14.461538,5.025641
2,Air Force,35,452,178,27,259,20.0,NaN,58,60,58,5.0,78,NaN,12.914286,5.085714
3,Alaska,34,505,183,24,305,NaN,NaN,62,61,74,20.0,88,NaN,14.852941,5.382353
4,Army,35,346,150,23,215,40.0,NaN,52,44,56,7.0,56,NaN,9.885714,4.285714


In [20]:

# Save the final DataFrames to CSV files with the TAG string
folder = '../data/tableau'

df_team_stats_final.to_csv(f'{folder}/penelties_by_team_{TAG}.csv', index=False)
df_ref_stats_updated.to_csv(f'{folder}/penelties_ref_stats_{TAG}.csv', index=False)



In [21]:
# SQL query to find the average count of penalties per referee per game
query_avg_count_penalty_per_ref = """
SELECT Referee, COUNT(*) / COUNT(DISTINCT Game_ID) AS Avg_Penalty_Count
FROM (
    SELECT Ref1 AS Referee, penalty_summary.Game_ID
    FROM game_details
    JOIN penalty_summary ON game_details.Game_ID = penalty_summary.Game_ID
    UNION ALL
    SELECT Ref2 AS Referee, penalty_summary.Game_ID
    FROM game_details
    JOIN penalty_summary ON game_details.Game_ID = penalty_summary.Game_ID
)
WHERE Referee IS NOT NULL
GROUP BY Referee
ORDER BY Avg_Penalty_Count DESC;
"""

# Execute the query and fetch results
cursor.execute(query_avg_count_penalty_per_ref)
avg_count_penalty_per_ref = cursor.fetchall()

# Show the results (top 10 referees by average penalty count)
avg_count_penalty_per_ref[:10]



[('Jake  Rukucki', 28),
 ('Sam Cucinotta', 19),
 ('Sweeney', 17),
 ('Joseph Causone', 17),
 ('JOHN  BOCHE', 17),
 ('Blanchard', 17),
 ('BRAD ALBERS', 17),
 ('Kenneth Gates', 16),
 ('Wall', 15),
 ('OBrien', 15)]

## Michigan State Specific queries below

In [22]:
# Update the function to fetch data related to Michigan State using the abbreviation "MSU"
def fetch_michigan_state_data(table_name, conn, abbreviation='MSU', full_name='Michigan State'):
    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name} WHERE Team IN ('{abbreviation}', '{full_name}');"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    return column_names, rows

# Connect to the database
conn = sqlite3.connect(db_path)

# Function to identify tables with a "Team" column
def has_team_column(table_name, conn):
    cursor = conn.cursor()
    query = f"PRAGMA table_info({table_name});"
    cursor.execute(query)
    columns = cursor.fetchall()
    column_names = [col[1] for col in columns]
    return 'Team' in column_names

# Connect to the database
conn = sqlite3.connect(db_path)

# Identify tables with a "Team" column
tables_with_team_column = [table for table in relevant_tables if has_team_column(table, conn)]

# Fetch data related to Michigan State from these tables
michigan_state_data = {}

for table in tables_with_team_column:
    column_names, rows = fetch_michigan_state_data(table, conn)
    michigan_state_data[table] = {
        'column_names': column_names,
        'rows': rows
    }


# Close the connection
conn.close()

# Show the number of records related to Michigan State in each table
{table: len(data['rows']) for table, data in michigan_state_data.items()}


NameError: name 'relevant_tables' is not defined

In [ ]:
# Function to fetch all data from a given table
def fetch_all_data(table_name, conn):
    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name};"
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [description[0] for description in cursor.description]
    return column_names, rows

# Connect to the database
conn = sqlite3.connect(db_path)

# Fetch all data from the advanced metrics tables
advanced_metrics_data = {}

for table in ['advanced_metrics_team1', 'advanced_metrics_team2']:
    column_names, rows = fetch_all_data(table, conn)
    advanced_metrics_data[table] = {
        'column_names': column_names,
        'rows': rows
    }

# Close the connection
conn.close()

# Convert the advanced metrics data into pandas DataFrames for easier analysis
advanced_metrics_dfs = {table: pd.DataFrame(data['rows'], columns=data['column_names']) 
                        for table, data in advanced_metrics_data.items()}

# Combine the two DataFrames into one for easier analysis
all_advanced_metrics_df = pd.concat(advanced_metrics_dfs.values(), ignore_index=True)

# Show the first few rows and columns of the combined DataFrame
# all_advanced_metrics_df.head(), all_advanced_metrics_df.columns.tolist()


In [ ]:
# Filter the data for Michigan State players
# We'll use the 'Game_ID' column to identify Michigan State games
msu_advanced_metrics_df = all_advanced_metrics_df[all_advanced_metrics_df['Game_ID'].str.contains('Michigan State|MSU', case=False, na=False)]

# Convert numerical columns to the proper data type for calculations
numeric_columns = msu_advanced_metrics_df.columns.difference(['Player', 'Game_ID'])
msu_advanced_metrics_df[numeric_columns] = msu_advanced_metrics_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Generate some basic summaries for the advanced metrics
advanced_summary_stats = {}

# Total Stats
advanced_summary_stats['Total Stats'] = msu_advanced_metrics_df.groupby('Player')[['TOTAL_Block', 'TOTAL_Miss', 'TOTAL_Saved', 'TOTAL_Goals', 'TOTAL_Total_Shots']].sum().reset_index()

# Even Strength Stats
advanced_summary_stats['Even Strength Stats'] = msu_advanced_metrics_df.groupby('Player')[['EVEN_Block', 'EVEN_Miss', 'EVEN_Saved', 'EVEN_Goals', 'EVEN_Total_Shots']].sum().reset_index()

# Power Play Stats
advanced_summary_stats['Power Play Stats'] = msu_advanced_metrics_df.groupby('Player')[['PP_Block', 'PP_Miss', 'PP_Saved', 'PP_Goals', 'PP_Total_Shots']].sum().reset_index()

# Close Game Stats
advanced_summary_stats['Close Game Stats'] = msu_advanced_metrics_df.groupby('Player')[['CLOSE_Block', 'CLOSE_Miss', 'CLOSE_Saved', 'CLOSE_Goals', 'CLOSE_Total_Shots']].sum().reset_index()

# Defensive Blocks and Faceoffs
advanced_summary_stats['Defensive & Faceoffs'] = msu_advanced_metrics_df.groupby('Player')[['D_Blocks', 'Faceoffs']].sum().reset_index()

# Show the summary stats
# advanced_summary_stats


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Prepare the data for visualization
viz_data = advanced_summary_stats['Total Stats'].copy()
viz_data = viz_data.melt(id_vars=['Player'], value_vars=['TOTAL_Block', 'TOTAL_Miss', 'TOTAL_Saved', 'TOTAL_Goals'], 
                         var_name='Metric', value_name='Count')

# Remove the 'TOTAL' row that summarizes the metrics, as it skews the graph
viz_data_filtered = viz_data[viz_data['Player'] != 'TOTAL']

# Calculate the sum of all metrics for each player to identify the top and bottom performers
total_counts = viz_data_filtered.groupby('Player')['Count'].sum().reset_index()
top_performers = total_counts.nlargest(10, 'Count')['Player'].tolist()

# Filter the data to only include the top performers
viz_data_top_performers = viz_data_filtered[viz_data_filtered['Player'].isin(top_performers)]

# Create the visualization
plt.figure(figsize=(15, 10))
sns.barplot(data=viz_data_top_performers, x='Player', y='Count', hue='Metric')

# Add data labels
for p in plt.gca().patches:
    if p.get_height() > -1:  # Only label the bars with height greater than 0
        plt.gca().annotate(f"{int(p.get_height())}", (p.get_x() + p.get_width() / 2., p.get_height()),
                           ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.title('Blocked vs Missed vs Saved vs Goals for Top Michigan State Players')
plt.xticks(rotation=45)
plt.show()
